# GPN MSA Dataset testing

Created the dataset class, now test to make sure it's correct and has no bugs and deals with all the weird things like RC correctly!

In [1]:
#so for this test, we will stick with 2 examples, let's find them
import pandas as pd
import os
base_path = '/data1/lesliec/sarthak'
seqs = pd.read_csv(os.path.join(base_path,'data/enformer/data/human/sequences.bed'), sep='\t', header=None)
seqs

,0,1,2,3
0,chr18,928386,1059458,train
1,chr4,113630947,113762019,train
2,chr11,18427720,18558792,train
3,chr16,85805681,85936753,train
4,chr3,158386188,158517260,train
...,...,...,...,...
38166,chr19,33204702,33335774,test
38167,chr14,41861379,41992451,test
38168,chr19,30681544,30812616,test
38169,chr14,61473198,61604270,test


In [2]:
split = 'train'
seqs_bed = seqs[seqs[3] == split]

In [3]:
extend = (196608-131072)//2
seqs_bed[1]-extend

0           895618
1        113598179
2         18394952
3         85772913
4        158353420
           ...    
34016     50482354
34017    135570001
34018    188971430
34019     10405331
34020    185697504
Name: 1, Length: 34021, dtype: int64

In [4]:
#let's find a value that is outside of the chromosome so seqs_bed[1]-extend is negative
#have to find which indexx

seqs_bed[seqs_bed[1]-extend < 0]

,0,1,2,3
1022,chr3,10000,141072,train
25870,chr4,10000,141072,train
27331,chr5,10000,141072,train
28360,chr16,10000,141072,train
30217,chr2,10000,141072,train
31591,chr12,10000,141072,train


In [5]:
seqs_bed.iloc[1022]

0      chr3
1     10000
2    141072
3     train
Name: 1022, dtype: object

In [6]:
#let's now get the actual value of the sequence, we'll do the label later
import zarr
zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'

zarr_open = zarr.open(zarr_path, mode='r')
zarr_open.tree()

/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/site-packages/zarr/core/group.py:2821: UserWarning: Object at .snakemake_timestamp is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


/
├── 1 (248956422, 100) |S1
├── 10 (133797422, 100) |S1
├── 11 (135086622, 100) |S1
├── 12 (133275309, 100) |S1
├── 13 (114364328, 100) |S1
├── 14 (107043718, 100) |S1
├── 15 (101991189, 100) |S1
├── 16 (90338345, 100) |S1
├── 17 (83257441, 100) |S1
├── 18 (80373285, 100) |S1
├── 19 (58617616, 100) |S1
├── 2 (242193529, 100) |S1
├── 20 (64444167, 100) |S1
├── 21 (46709983, 100) |S1
├── 22 (50818468, 100) |S1
├── 3 (198295559, 100) |S1
├── 4 (190214555, 100) |S1
├── 5 (181538259, 100) |S1
├── 6 (170805979, 100) |S1
├── 7 (159345973, 100) |S1
├── 8 (145138636, 100) |S1
├── 9 (138394717, 100) |S1
├── X (156040895, 100) |S1
└── Y (57227415, 100) |S1

In [7]:
temp_out = zarr_open['3'][0:141072+extend]
temp_out

array([[b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       [b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       [b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       ...,
       [b'A', b'A', b'A', ..., b'-', b'-', b'-'],
       [b'A', b'A', b'A', ..., b'-', b'-', b'-'],
       [b'A', b'A', b'A', ..., b'-', b'-', b'-']], dtype='|S1')

In [8]:
temp_out.shape

(173840, 100)

In [9]:
#have to append like 20000 Ns, but it's fine, we can ignore it for now, let's just see if it matches our dataset
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)

base_path: /data1/lesliec/sarthak/


In [10]:
out = dataset[1022]

In [11]:
out[0].shape, out[1].shape #that's the proper shape for input to a cnn!

(torch.Size([500, 196608]), torch.Size([1792, 674]))

In [12]:
out[0]

tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [13]:
temp_out[:,0]

array([b'n', b'n', b'n', ..., b'A', b'A', b'A'], dtype='|S1')

In [14]:
temp_out[-10:,0]

array([b'T', b'T', b'T', b'C', b'C', b'T', b'A', b'A', b'A', b'A'],
      dtype='|S1')

In [15]:
#the last 10 should be like this
out[0][:5,-10:] #wait this looks exxaclty correct...

tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
#let's look at a different animal

temp_out[-10:,40]

array([b'T', b'T', b'T', b'C', b'C', b'C', b'A', b'A', b'A', b'A'],
      dtype='|S1')

In [17]:
out[0][5*40:5*41,-10:] #again this is pretty good...

tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
#the final thing to look at is rc aug

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64, rc_aug=True)

In [25]:
out = dataset[1022]
out[0]

tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [26]:
#so once for identical, once for rc aug 
out_rc = dataset[1022]
out_rc[0] #it's different so assume they are indeed different

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [28]:
out[1][:,0]

tensor([0.1244, 0.1235, 0.0645,  ..., 0.0358, 0.0175, 0.0378])

In [29]:
out_rc[1][:,0] #indeed looks like it's flipped properly!!

tensor([0.0378, 0.0175, 0.0358,  ..., 0.0645, 0.1235, 0.1244])

In [30]:
out[0].shape

torch.Size([500, 196608])

In [32]:
import torch
torch.allclose(out[0], out_rc[0])

False

In [33]:
#obviously one is the RC, now let's manually do an RC and check?? or just check some random parts
out[0][:5,:10]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [34]:
out_rc[0][:5,-10:]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [39]:
out[0][:5,100000:100010]

tensor([[0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [40]:
out_rc[0][:5,-100010:-100000]

tensor([[0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
#very clealry these are the reverse complement of each other, I'm just going to assume it's right for now, seems good??

#we should also check to make sure this out itself is correct, can check if the RC is correct too!!

In [41]:
#compare with the zarr values! this can be slow, it's a manual check!! should also check all the speices?
temp_out.shape

(173840, 100)

In [42]:
#first append on a bunch of Ns
temp_out

array([[b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       [b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       [b'n', b'-', b'-', ..., b'-', b'-', b'-'],
       ...,
       [b'A', b'A', b'A', ..., b'-', b'-', b'-'],
       [b'A', b'A', b'A', ..., b'-', b'-', b'-'],
       [b'A', b'A', b'A', ..., b'-', b'-', b'-']], dtype='|S1')

In [43]:
import numpy as np
decoded_out = np.char.decode(temp_out)
decoded_out

array([['n', '-', '-', ..., '-', '-', '-'],
       ['n', '-', '-', ..., '-', '-', '-'],
       ['n', '-', '-', ..., '-', '-', '-'],
       ...,
       ['A', 'A', 'A', ..., '-', '-', '-'],
       ['A', 'A', 'A', ..., '-', '-', '-'],
       ['A', 'A', 'A', ..., '-', '-', '-']], dtype='<U1')

In [44]:
196608-173840

22768

In [45]:
#make an array that is 22768 x 100 of ns
ns_array = np.full((22768, 100), 'N')
ns_array

array([['N', 'N', 'N', ..., 'N', 'N', 'N'],
       ['N', 'N', 'N', ..., 'N', 'N', 'N'],
       ['N', 'N', 'N', ..., 'N', 'N', 'N'],
       ...,
       ['N', 'N', 'N', ..., 'N', 'N', 'N'],
       ['N', 'N', 'N', ..., 'N', 'N', 'N'],
       ['N', 'N', 'N', ..., 'N', 'N', 'N']], dtype='<U1')

In [47]:
ns_array.shape, decoded_out.shape

((22768, 100), (173840, 100))

In [48]:
#now combine
full_out = np.concatenate((ns_array, decoded_out), axis=0)
full_out.shape

(196608, 100)

In [51]:
full_out = np.char.lower(full_out)

In [49]:
key = {'a': 0, 'c': 1, 'g': 2, 't': 3, 'n': 4, '-': 4}
final_zarr = np.zeros((full_out.shape[0], full_out.shape[1], 5), dtype=np.int8)
final_zarr.shape

(196608, 100, 5)

In [52]:
for i in range(full_out.shape[0]):
    for j in range(full_out.shape[1]):
        final_zarr[i,j,key[full_out[i,j]]] = 1

In [55]:
#let's verify just one random one
final_zarr[:,0,:]

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int8)

In [61]:
torch.tensor(final_zarr[:,0,:])

tensor([[0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        ...,
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=torch.int8)

In [63]:
out[0].shape

torch.Size([500, 196608])

In [69]:
torch.tensor(final_zarr[:,0,:].T).shape

torch.Size([5, 196608])

In [66]:
out[0][:5,:].shape

torch.Size([5, 196608])

In [70]:
torch.allclose(torch.tensor(final_zarr[:,0,:].T).float(), out[0][:5,:])

True

In [71]:
#ok that matches perfect, let's try a different species
torch.allclose(torch.tensor(final_zarr[:,80,:].T).float(), out[0][5*80:5*81,:])

True

In [72]:
torch.allclose(torch.tensor(final_zarr[:,80,:].T).float(), out[0][:5,:])

False

In [73]:
#ok man this seems to actually work!!! 
#let's just assume that the RC works, for what I've seen, this seems quite solid!!
#let's finally time it
from tqdm import tqdm
dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
for i in tqdm(range(100)):
    out = dataset[i]

 29%|██▉       | 29/100 [00:59<02:25,  2.05s/it]


KeyboardInterrupt: 

# speeding up with dataloader

In [3]:
#almost 2 s per iteration...a bit slow but we can't really speed it up that much, the data loading will be slow, maybe can add more cores, try data loader
#let's try with 8 workers
import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 0/100 [00:40<?, ?it/s]


KeyboardInterrupt: 

In [1]:
#huh so I have too many workers and it says it will freeze? Let's request more cores? If that's the issue that would make sense...
import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


base_path: /data1/lesliec/sarthak/


  0%|          | 0/100 [03:09<?, ?it/s]


KeyboardInterrupt: 

In [2]:
import os
print("os.cpu_count()", os.cpu_count())
import torch
print("torch.get_num_threads()", torch.get_num_threads())

os.cpu_count() 64
torch.get_num_threads() 1


In [3]:
torch.set_num_threads(8)
torch.set_num_interop_threads(8)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 0/100 [00:46<?, ?it/s]


KeyboardInterrupt: 

In [4]:
print("torch.get_num_threads()", torch.get_num_threads())

torch.get_num_threads() 8


In [1]:
#now it's 8, but that doesn't seem to solve anything

import os
for k, v in sorted(os.environ.items()):
    if 'THREAD' in k or 'CPU' in k:
        print(k, "=", v)


SLURM_CPUS_ON_NODE = 8
SLURM_CPUS_PER_TASK = 8
SLURM_CPU_BIND = quiet,mask_cpu:0x0001F80300000000
SLURM_CPU_BIND_LIST = 0x0001F80300000000
SLURM_CPU_BIND_TYPE = mask_cpu:
SLURM_CPU_BIND_VERBOSE = quiet
SLURM_JOB_CPUS_PER_NODE = 1


In [3]:
#ok I reset it and now we have 8 threads I think...
import torch
print("torch.get_num_threads()", torch.get_num_threads())

torch.get_num_threads() 8


In [4]:
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

base_path: /data1/lesliec/sarthak/


  0%|          | 0/100 [01:06<?, ?it/s]


KeyboardInterrupt: 

In [3]:
#ok so now we don't get the warning, but it seems like we're not actually getting 8 cores... what if we request it from the login node??
#ok made it on the login node and didn't get that warning? but still it doesn't seem like it's actually working, what if we just did salloc without the no shell and see what happens, worth trying because at this rate nothing is working...
#how to parallelize
import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [04:19<?, ?it/s]


KeyboardInterrupt: 

In [2]:
#ok this also doesn't seem to work for some reason??

import os
for k, v in sorted(os.environ.items()):
    if 'THREAD' in k or 'CPU' in k:
        print(k, "=", v)

SLURM_CPUS_ON_NODE = 8
SLURM_CPU_BIND = quiet,mask_cpu:0x000007F800000000
SLURM_CPU_BIND_LIST = 0x000007F800000000
SLURM_CPU_BIND_TYPE = mask_cpu:
SLURM_CPU_BIND_VERBOSE = quiet
SLURM_JOB_CPUS_PER_NODE = 8


In [6]:
#let's see

from torch.utils.data import Dataset
import torch

class DummyDataset(Dataset):
    def __getitem__(self, idx):
        return torch.rand(10), idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])


torch.Size([8, 10]) tensor([0, 1, 2, 3, 4, 5, 6, 7])
torch.Size([8, 10]) tensor([ 8,  9, 10, 11, 12, 13, 14, 15])
torch.Size([8, 10]) tensor([16, 17, 18, 19, 20, 21, 22, 23])
torch.Size([8, 10]) tensor([24, 25, 26, 27, 28, 29, 30, 31])
torch.Size([8, 10]) tensor([32, 33, 34, 35, 36, 37, 38, 39])
torch.Size([8, 10]) tensor([40, 41, 42, 43, 44, 45, 46, 47])
torch.Size([8, 10]) tensor([48, 49, 50, 51, 52, 53, 54, 55])
torch.Size([8, 10]) tensor([56, 57, 58, 59, 60, 61, 62, 63])
torch.Size([8, 10]) tensor([64, 65, 66, 67, 68, 69, 70, 71])
torch.Size([8, 10]) tensor([72, 73, 74, 75, 76, 77, 78, 79])
torch.Size([8, 10]) tensor([80, 81, 82, 83, 84, 85, 86, 87])
torch.Size([8, 10]) tensor([88, 89, 90, 91, 92, 93, 94, 95])
torch.Size([8, 10]) tensor([ 96,  97,  98,  99, 100, 101, 102, 103])
torch.Size([8, 10]) tensor([104, 105, 106, 107, 108, 109, 110, 111])
torch.Size([8, 10]) tensor([112, 113, 114, 115, 116, 117, 118, 119])
torch.Size([8, 10]) tensor([120, 121, 122, 123, 124, 125, 126, 127])


In [7]:
#let's just use zarr in this dummy dataset
import zarr
zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
zarr_open = zarr.open(zarr_path, mode='r')

class DummyDataset(Dataset):
    def __getitem__(self, idx):
        return zarr_open['1'][:10], idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

KeyboardInterrupt: 

In [9]:
#what if we open zarr in get item?
# zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
# zarr_open = zarr.open(zarr_path, mode='r')

class DummyDataset(Dataset):
    def __getitem__(self, idx):
        zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
        zarr_open = zarr.open(zarr_path, mode='r')
        return zarr_open['1'][:10], idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

KeyboardInterrupt: 

In [10]:
zarr_open['1'][:10]

array([[b'n', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-'],
       [b'n', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-', b'-',
        b'-', b'-', b'-

In [11]:
class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
    
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        z_store = zarr.open(self.zarr_path, mode='r')
        return z_store['1'][:10], idx
    
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])
#huh it's something weird with this...

KeyboardInterrupt: 

In [1]:
#so zarr can easily access it but not in parallel...
#let's reset it and try this mp thing

import multiprocessing as mp
mp.set_start_method("spawn", force=True)

import zarr
import torch
from torch.utils.data import Dataset, DataLoader

class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
    
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        z_store = zarr.open(self.zarr_path, mode='r')
        return z_store['1'][:10], idx
    
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^

RuntimeError: DataLoader worker (pid(s) 3266081, 3266082, 3266083, 3266084) exited unexpectedly

In [1]:
#ok so the issue is I need to create something I can import wtf?
#made a dummy dataset.py file

import multiprocessing as mp
mp.set_start_method("spawn", force=True)

from code_test.dummy_dataset import DummyDataset
from torch.utils.data import DataLoader

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])


[[b'n' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-']
 [b'n' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'
  b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-' b'-'

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
#ok so this works on a jupyter notebook, what about a straight up python script?
#maybe we don't need to touch multiprocessing in that case?
import zarr
zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
zarr_open = zarr.open(zarr_path, mode='r')
out = zarr_open['1'][:10]

In [5]:
out.shape

(10, 100)

In [7]:
import numpy as np
np.char.decode(out)

array([['n', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['n', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
#let' smake a new class to test the full limits of what we can do in python straight up

import zarr
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        z_store = zarr.open(self.zarr_path, mode='r')
        return len(z_store['1'][:10]), idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

In [10]:
#that works like a charm???

#let's try this code now

#but this doesn't work?

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#perhaps because the open zarr is not defined in each, let's change that? Now it should work

# found potential fix

In [1]:
#so there's a difference between tasks and cpus, let's see if this can help!!

#first let's see what OS says, we not connecting to external core or anythin

import os
print("os.cpu_count()", os.cpu_count())
import torch
print("torch.get_num_threads()", torch.get_num_threads())

os.cpu_count() 64
torch.get_num_threads() 60


In [2]:
import os
for k, v in sorted(os.environ.items()):
    if 'THREAD' in k or 'CPU' in k:
        print(k, "=", v)

#huh, I guess we just are getting full access to the whole node because we sshd directly? idk

In [4]:
import zarr
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
        self.z_store = zarr.open(self.zarr_path, mode='r')
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        # z_store = zarr.open(self.zarr_path, mode='r')
        return len(self.z_store['1'][:10]), idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=1, num_workers=4)
print('now loading')
for batch in loader:
    print(batch[0].shape, batch[1])

now loading


KeyboardInterrupt: 

In [5]:
#yeah so that still isn't working...

import zarr
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        z_store = zarr.open(self.zarr_path, mode='r')
        return len(z_store['1'][:10]), idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

KeyboardInterrupt: 

In [ ]:
#however that does work within the terminal, so it seems that htis works, what about the first one tho?
#that doesn't work, so if we open it in the init it doesn't work, it does work if we open it in the get item tho

#final thing is let's try openingi t in both

import zarr
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DummyDataset(Dataset):
    def __init__(self):
        self.zarr_path = '/data1/lesliec/sarthak/data/gpn/99.zarr'
        # z_store = zarr.open(self.zarr_path, mode='r') #if set as self, it doesn't really work?
        # print(z_store['1'].shape, 'init')
    def __getitem__(self, idx):
        # Open inside __getitem__ (or __init__ of each worker)
        z_store = zarr.open(self.zarr_path, mode='r')
        return len(z_store['1'][:10]), idx
    def __len__(self):
        return 1000

ds = DummyDataset()
loader = DataLoader(ds, batch_size=8, num_workers=4)
for batch in loader:
    print(batch[0].shape, batch[1])

In [ ]:
#yeah we can't open zarr in the init at all, I guess it conflicts??
#but opening it in the get item seems to be fine!
#now the real test is if my dataset works, let's test it

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#it works but is like 20 s per iteration... why is it so slow??

#speeds up a bit to 15 it/s

#oh wait my batch size is 1, so it's entirely doing just 1 workers worth?

In [ ]:
#with num workers 0

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#this is like 3 seconds

In [ ]:
#just doing the dataset

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)

for i in tqdm(range(100)):
    out = dataset[i]
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#this too is about 3 seconds per iteration, why is this so much faster?

In [ ]:
#why does it get slow when we have more workers???

#final test let's increase batch size!

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#if this takes much longer than 3*8 seconds, then there's definitely some issue...
#it takes 50 s per iteration...

In [ ]:
#let's make them persistent

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4, persistent_workers=True)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#persistent is somehow slower, now first iteration is 20 seconds, nexxt ones are 10 s??

In [6]:
import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=0)
for i in tqdm(range(100)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#this is 9 s per iteration lol. So I guess we can't speed up the parallelization at all...

KeyboardInterrupt: 

In [8]:
#so let's see how long roughly it would take for the full train set just loading it

import torch
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)
# print(len(dataloader)) #34021 obviously
for i in tqdm(range(34021)):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

34021


In [12]:
#even at 1s per iteration it can take up to 10 hours for the full dataset... but we can't really speed this up? I don't know... maybe don't use zarr?
#it's fine I guess, just use 1 worker...

#let's see if it's zarr that's the issue and not able to concurrently access
import zarr
import time
import multiprocessing
import numpy as np

# Path to your Zarr store
ZARR_PATH = "/data1/lesliec/sarthak/data/gpn/99.zarr"

# Open store in the main process just to get shape info
z_in = zarr.open(ZARR_PATH, mode='r')['1']

# Suppose we read along the first dimension in random slices
shape = z_in.shape  # e.g. (N, X, Y, ...)
indices_to_read = np.random.randint(low=0, high=shape[0], size=1000)

def read_chunk(idx):
    """
    Each worker process runs this function to read a chunk of data from the Zarr store.
    """
    # Open zarr store *inside* worker to avoid pickling overhead
    # or interfering with other processes:
    z_worker = zarr.open(ZARR_PATH, mode='r')
    # Start timing
    t0 = time.time()
    # Read a small or moderate slice – e.g. single slice
    data = z_worker[idx]  # shape might be (X, Y, ...)
    dt = time.time() - t0
    return dt

def parallel_read_test(indices, num_workers=8):
    # Use a pool of processes
    pool = multiprocessing.Pool(num_workers)
    t0 = time.time()
    # Map all the indices to the workers
    durations = pool.map(read_chunk, indices)
    total_time = time.time() - t0
    pool.close()
    pool.join()
    return total_time, durations

def sequential_read_test(indices):
    """
    Same read, but in a single process (sequential) to get a baseline.
    """
    z_seq = zarr.open(ZARR_PATH, mode='r')['1']
    t0 = time.time()
    durations = []
    for idx in indices:
        t_start = time.time()
        _ = z_seq[idx]
        durations.append(time.time() - t_start)
    total_time = time.time() - t0
    return total_time, durations


# (1) Test sequential read
seq_total_time, seq_durations = sequential_read_test(indices_to_read)
print(f"[SEQUENTIAL] Total time: {seq_total_time:.2f} s")
print(f"[SEQUENTIAL] Avg time per read: {np.mean(seq_durations):.4f} s")

# (2) Test parallel read
par_total_time, par_durations = parallel_read_test(indices_to_read, num_workers=8)
print(f"[PARALLEL x8] Total time: {par_total_time:.2f} s")
print(f"[PARALLEL x8] Avg time per read: {np.mean(par_durations):.4f} s")

#print(f"[SEQUENTIAL] Total time: {seq_total_time:.2f} s")
# [SEQUENTIAL] Total time: 0.74 s
# >>> print(f"[SEQUENTIAL] Avg time per read: {np.mean(seq_durations):.4f} s")
# [SEQUENTIAL] Avg time per read: 0.0007 s

#parallel just isn't even running... oh well dude.

[SEQUENTIAL] Total time: 0.80 s
[SEQUENTIAL] Avg time per read: 0.0008 s


Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-17:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/mult

KeyboardInterrupt: 

  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py

  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/data1/lesliec/sarthak/caduceus/.pixi/envs/default/lib/python3.11/multiprocessing/pool.py", 

In [ ]:
#the final options might be the zarr version? Let's test that... see the version of zarr used by gpn and also their dataloader
#just changed the version let's see, use our old version
#this seems like it might actually work, let's try again

import torch
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
# print(len(dataloader)) #34021 obviously
for i in tqdm(range(len(dataloader))):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)
        
#not apples to apples as we can't load labels since it's zarr 3 but let's see
#wait this is actually it, took like 3 seconds man!!!! 
#still will take 4.5 hours but that's an improvement, maybe cuz not loading labels tho?
#3.99 s/it

In [ ]:
#see with num workers = 0

import torch
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=0)
# print(len(dataloader)) #34021 obviously
for i in tqdm(range(len(dataloader))):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#actually a similar speed, but that's at least partiaally due to us not using the init. Let's use the old one
#4.26 s/it

In [ ]:
import torch
import src.dataloaders.datasets.GPNMSA_dataset_noparallel as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
# print(len(dataloader)) #34021 obviously
for i in tqdm(range(len(dataloader))):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)

#this new one just seem better, regardless good to stick to zarr 2 for this exxact reason!! at least no slowdown if num workers is 1. Idk why zarr 2 does this, probably the other version information?

In [ ]:
#let's go make a job to make this zarr
import zarr
zarr.__version__

In [ ]:
#let's just make num workers 0 and use that for now
import torch
import src.dataloaders.datasets.GPNMSA_dataset as d
from tqdm import tqdm

dataset = d.GPNMSADataset('train', 196608, cell_type='DNase', one_hot=True, data_path='/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr', pool = 64)
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=False, num_workers=0)
# print(len(dataloader)) #34021 obviously
for i in tqdm(range(len(dataloader))):
    out = next(iter(dataloader))
    if i == 0:
        print(out[0].shape)
        print(out[1].shape)
#only takes 3 hours, not bad tbh. Wait that's after we removed the sstuff and downgraded the zarr version...
#when we fix it again to do the full thing, probalby back to the 10 hours?